# Gender and Age Detection 

In [15]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Dropout, BatchNormalization, Flatten, Input
from sklearn.model_selection import train_test_split

In [18]:
# Defining the path .
datasetFolder = r"C:\Users\ACER\Documents\Gender Detection\DataSets\UTKFace"

In [3]:
# Creating empty list.

pixels = []
age = []
gender = []
for img in os.listdir(datasetFolder) : # os.listdir opens the directory "datasetFolder"
    # Label of each image is splitted on "_" and required information is stored in required variable.
    ages = img.split("_")[0] 
    genders = img.split("_")[1]
    img = cv2.imread(str(datasetFolder) + "/" + str(img)) # Reading each image from the path of folder provided.
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Converting the input image from BGR to RGB as computer by default sees an image in BGR.
    
    # Appending necessary data in respective created lists.
    pixels.append(np.array(img))
    age.append(np.array(ages))
    gender.append(np.array(genders))

# Converting list to array
age = np.array(age, dtype = np.int64) 
pixels = np.array(pixels)
gender = np.array(gender, np.uint64)

In [4]:
# Printing the length of the pixel .
p = len(pixels)
print(f"No. of images working upon {p}")

No. of images working upon 23708


In [5]:
# Splitting the images in train and test dataset.
x_train, x_test, y_train, y_test = train_test_split(pixels, age, random_state = 100)

# Splitting the dataset in train and test dataset as gender as.
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(pixels, gender, random_state = 100)

In [6]:
# Checking the shape of the images set. Here (200, 200, 3) are height, width and channel of the images respectively.
x_train.shape, x_train_2.shape, x_test.shape, x_test_2.shape, 

((17781, 200, 200, 3),
 (17781, 200, 200, 3),
 (5927, 200, 200, 3),
 (5927, 200, 200, 3))

In [7]:
# Checking the shape of the target variable.
y_train.shape, y_train_2.shape, y_test.shape, y_test_2.shape

((17781,), (17781,), (5927,), (5927,))

###### Below cell of code is used to create layers of a convolution neural network model. The layers in a CNN model are : 
* Input Layer
* Convolution Layer
* ReLu Layer
* Pooling Layer
* Fully Connected Network

In [8]:
inputLayer = Input(shape = (200, 200, 3)) # From the Input Model called from keras.models. Again (200, 200, 3) are height, width and channel of the images respectively.
convLayer1 = Conv2D(140,(3,3), activation = 'relu')(inputLayer) 
'''An activation function is basically just a simple function that transforms its inputs into outputs that have a certain range.
Also the ReLu activation transforms the -ve vaulues into 0 and positive remains the same, hence it is known as half rectifier as
well.'''
convLayer2 = Conv2D(130,(3,3), activation = 'relu')(convLayer1) # Creating seccond layer of CNN.
batch1 = BatchNormalization()(convLayer2) # Normalizing the data.
poolLayer3 = MaxPool2D((2,2))(batch1) # Creating third, Pool Layer of the CNN.
convLayer3 = Conv2D(120,(3,3), activation = 'relu')(poolLayer3) # Adding the third Layer.
batch2 = BatchNormalization()(convLayer3) # Normalizing the layer.
poolLayer4 = MaxPool2D((2,2))(batch2) #Adding fourth layer of CNN. 
flt = Flatten()(poolLayer4) # Flattening the data.

In [9]:
age_model = Dense(128,activation="relu")(flt) # Here 128 is the no. of neurons connected with the flatten data layer.
age_model = Dense(64,activation="relu")(age_model) #Now as we move down, no. of neurons are reducing with previous neurons connected to them.
age_model = Dense(32,activation="relu")(age_model) 
age_model = Dense(1,activation="relu")(age_model)

In [10]:
gender_model = Dense(128,activation="relu")(flt) # The same work as above with 128 neurons is done for gender predictive model.
gender_model = Dense(80,activation="relu")(gender_model)
gender_model = Dense(64,activation="relu")(gender_model)
gender_model = Dense(32,activation="relu")(gender_model)
gender_model = Dropout(0.5)(gender_model) # Drop-out layer is added to dodge the overfitting of the model.
'''Softmax is a mathematical function that converts a vector of numbers into a vector of probabilities, where the probabilities 
of each value are proportional to the relative scale of each value in the vector. Here it is used as an activation function.'''
gender_model = Dense(2,activation="softmax")(gender_model) 

'Softmax is a mathematical function that converts a vector of numbers into a vector of probabilities, where the probabilities \nof each value are proportional to the relative scale of each value in the vector. Here it is used as an activation function.'

###### Below cell of code is to make an object of the Model from keras.models.

In [11]:
model = Model(inputs=inputLayer,outputs=[age_model,gender_model]) # Adding the input layer and the output layer in our model and making the object.
model.compile(optimizer="adam",loss=["mse","sparse_categorical_crossentropy"],metrics=['mae','accuracy']) 
model.summary() # To get the summary of our model.

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 198, 198, 140 3920        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 196, 196, 130 163930      conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 196, 196, 130 520         conv2d_1[0][0]                   
_______________________________________________________________________________________

In [ ]:
save = model.fit(x_train,[y_train,y_train_2], validation_data=(x_test,[y_test,y_test_2]),epochs=50)
model.save("model.h5")

Epoch 1/50
